# Laboratorium 1 - Convolutional Neural Network cz. I



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Celem pierwszych zajęć jest instalacja bibliotek używanych na zajęciach oraz wprowadzenie do języka Python. Zajęcia będą przedstawiane w formacie notatników Jupyter. Każda osoba ma możliwość używania dowolnych narzędzi i systemu operacyjnego z którymi czuje się komfortowo. 

Rekomendowane biblioteki:

*   [PyTorch](https://pytorch.org/docs/stable/index.html)
*   [Keras](https://keras.io/api/)
*   [PyTorch Geometrics](https://pytorch-geometric.readthedocs.io/en/latest/)



# *Zadanie 1*.


Po zapoznaniu się z obsluga interfejsu Google Colab. **Zaimplementuj** trzy wielowarstwowe Convolutional Neural Networks (CNNs) do klasyfikacji datasetów:
* [MNIST](https://paperswithcode.com/dataset/mnist)
* [Fashion MNIST](https://paperswithcode.com/dataset/fashion-mnist)
* [CIFAR-10](https://paperswithcode.com/dataset/cifar-10)

W tym celu wykorzystaj biblioteke **Keras** albo **PyTorch**.

Ponizej przyklad jak wczytac dane w **Keras**

```
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
```

# *Wprowadzenie do CNN*

Pierwsza warstwa zawsze musi posiadać parametr input_shape, który mówi o tym jakie dane dostanie sieć neuronowa na początku. **Input Layer** Jest to warstwa wejściowa, która reprezentuje obraz wejściowy do sieci. Warto zwrócić uwagę na rodzaj obrazu, który mamy. W przypadku kolorowych zdjęć będziemy używać trzech kanałów wejściowych RGB, odpowiadającym odpowiednio kanałom czerwonym, zielonym i niebieskim. W naszym przypadku (Fashion Mnist) mamy skalę szarości i dlatego mamy jeden kanał.

**Warstwa wejściowa (Input Layer)**
Jest to warstwa wejściowa, która reprezentuje obraz wejściowy do sieci. Warto zwrócić uwagę na rodzaj obrazu, który mamy. W przypadku kolorowych zdjęć będziemy używać trzech kanałów wejściowych RGB, odpowiadającym odpowiednio kanałom czerwonym, zielonym i niebieskim. W naszym przypadku (Fashion Mnist) mamy skalę szarości i dlatego mamy jeden kanał.

Warstwa wejsciowa jako Conv2D:
```
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
```

**Warstwa konwolucyjna / splotowa (Convolutional Layers)**

Warstwy splotowe są podstawą sieci konwolucyjnych CNN, ponieważ zawierają wyuczone filtry (kernele), które wyodrębniają cechy odróżniające od siebie różne obrazy.

Kiedyś zastanawiałem się, dlaczego definiując warstwę CNN nie podajemy tych filtrów. Tak naprawdę wartości w filtrach (tak jak wyżej sami mogliście sobie dobrać, jakie chcieliście) są dobierane i optymalizowane podczas trenowania sieci. I są dobrane tak, by minimalizować błąd naszej sieci przy rozwiązywaniu problemu, który zdefiniujemy.

**Kernel Size (rozmiar jądra)**
Rozmiar jądra, często nazywany również rozmiarem filtra, odnosi się do wymiarów przesuwanego okna nad wejściem. Wybór tego hiperparametru ma ogromny wpływ na zadanie klasyfikacji obrazu. Na przykład małe jądra są w stanie wydobyć z danych wejściowych znacznie większą ilość informacji zawierających wysoce lokalne funkcje. Mniejszy rozmiar jądra prowadzi również do mniejszego zmniejszenia wymiarów warstw, co pozwala na głębszą architekturę.

**Padding (wypełnienie)**
Padding definiuje sposób obsługi obramowania próbki. Umożliwia to otrzymanie rozmiaru wyjścia takiego samego jak rozmiar wejścia (przy założeniu, że strides jest przesunięciem o jeden). Osiąga się to kosztem dodania dodatkowych (sztucznych) wag na krawędziach (najczęściej z wartością zero). Dlaczego z zerami? Bo są wydajne – zapewniają prostotę i wydajność obliczeniową. 


**Strides (kroki)**
Parametr stride wskazuje, o ile pikseli jądro powinno zostać przesunięte na raz. Czyli inaczej mówiąc, oznacza krok przesunięcia okna filtra. Najczęściej używa się kroku wynoszącego 1 dla warstw splotowych. Oznacza to, że ​​iloczyn skalarny jest wykonywany w oknie wejściowym np. 3×3 w celu uzyskania wartości wyjściowej, a następnie jest przesuwany o jeden piksel dla każdej kolejnej operacji.

```
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
```


**Warstwa Flatten**

Jest to ważny krok. Ta warstwa przekształca naszą wielowymiarową warstwę w sieci w jednowymiarowy wektor. Robimy to po to, aby dopasować dane wejściowe w pełni połączonej warstwy do klasyfikacji. Na przykład tensor o wielkości 10x10x3 zostałby przekształcony w wektor o rozmiarze 300 (1 x 300).

Dlaczego to robimy? Bo zadaniem poprzedzających warstw splotowych sieci było wyodrębnienie cechy z obrazu wejściowego. A teraz nadszedł najwyższy czas, aby sklasyfikować cechy.

Najczęściej używamy funkcji softmax do klasyfikowania tych cech, co wymaga jednowymiarowych danych wejściowych. Dlatego konieczna jest spłaszczona warstwa.

```
model.add(Flatten())
```

**Warstwa Dense**
Warstwę gęstą (ang. dense layer lub fully connected layer), w której wszystkie neurony z warstwy wcześniejszej są połączone ze wszystkimi neuronami z warstwy kolejnej. Warstwy te (oraz ewentualnie inne, przystosowane do konkretnego zagadnienia) mogą występować w sieci wielokrotnie, na różnych poziomach, z różnymi konfiguracjami. Zaprojektowanie dobrej sieci neuronowej jest kluczowym czynnikiem decydującym o jej skuteczności i szybkości trenowania i działania.

**Warstwy poolingu**  to warstwy które wykorzystuje się w sieciach neuronowych do zmniejszania wymiarowości przestrzeni cech. W Keras dostępne są dwie podstawowe warstwy poolingu:

1. MaxPooling2D - wykonuje operację poolingu z użyciem największej wartości na obszarze próbkowania. Ta warstwa jest powszechnie używana w sieciach konwolucyjnych i pomaga zmniejszyć rozmiar map cech.

2. AveragePooling2D - wykonuje operację poolingu poprzez uśrednienie wartości na obszarze próbkowania. Ta warstwa jest również często stosowana w sieciach konwolucyjnych, szczególnie gdy chcemy zmniejszyć rozmiar map cech, ale nie chcemy tracić tak wiele informacji jak w przypadku warstwy MaxPooling2D.

Obie warstwy poolingu mają parametry, takie jak rozmiar okna próbkowania i krok, które pozwalają na dostosowanie sposobu, w jaki jest przeprowadzane próbkowanie.

Idea poolingu polega na tym, że kilka pikseli np. rozmiar 2×2 mapujemy na 1 piksel. Najbardziej znane są dwa rodzaje poolingów:

* max (maximum) – gdy bierzemy maksymalną wartość z 4 pikseli,
* avg (avarage) – gdy bierzemy średnią wartość z 4 pikseli.

```
model.add(MaxPool2D(pool_size=(2, 2)))
```

**Funkcje aktywacji**

O różnych funkcjach aktywacji przeczytaj -> [klikając tutaj](https://laptrinhx.com/complete-guide-of-activation-functions-574622854/)

```
model.add(Dense(10, activation = 'softmax'))
```

**Funkcje straty**

* dla klasyfikacji binarnej użycie binarnej entropii krzyżowej (binary_crossentropy),
* dla klasyfikacji wielowymiarowej użycie kategoryzującej entropii krzyżowej (categorical _crossentropy),
* w przypadku regresji błąd średniokwadratowy (mse).


``` 
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
```

**Optymalizatory**

Optymalizator — mechanizm dostrajania sieci na podstawie danych zwracanych przez funkcję straty. Strata jest wartością, którą sieć będzie próbowała minimalizować podczas trenowania,
a więc powinna być miarą sukcesu w rozwiązaniu danego problemu.
Optymalizator określa dokładnie sposób używania gradientu straty podczas aktualizacji parametrów: w sieci neuronowej możesz zastosować np. algorytm optymalizatora RMSProp czy algorytm stochastycznego spadku wzdłuż gradientu z pędem.

Konfiguracja optymalizacji — z jakiego optymalizatora będziesz korzystać?
Jaki współczynnik uczenia wybierzesz? W większości sytuacji bezpiecznym wyborem jest optymalizator rmsprop i domyślna wartość jego współczynnika uczenia. 

1. Gradient Descent - jest to najprostszy optymalizator, który polega na iteracyjnej aktualizacji wag na podstawie wartości gradientu funkcji straty. Można wyróżnić kilka wariantów Gradient Descent, m.in. Batch Gradient Descent, Stochastic Gradient Descent oraz Mini-Batch Gradient Descent.

2. Momentum - optymalizator ten dodaje do aktualizacji wag informacje o poprzedniej iteracji, co ma na celu przyspieszenie procesu uczenia oraz zapobieganie utknięciu w minimach lokalnych funkcji straty.

3. Adagrad - optymalizator ten dostosowuje tempo uczenia dla każdej wagi na podstawie historycznych wartości gradientu. Wagi, które często się aktualizują, mają mniejsze tempo uczenia, a te, które są aktualizowane rzadziej, mają większe tempo uczenia.

4. Adadelta - optymalizator ten jest podobny do Adagrad, ale zamiast wykorzystywać historyczne wartości gradientu, korzysta z historycznych wartości kwadratów aktualizacji wag. Ma to na celu przeciwdziałanie problemowi zanikającego lub eksplodującego gradientu.

5. Adam - optymalizator ten łączy cechy Momentum i Adagrad. Wprowadza również nowe pojęcie estymacji momentów, które pozwala na adaptacyjną zmianę tempa uczenia w zależności od wartości gradientu oraz historycznych wartości.

6. RMSProp - optymalizator ten polega na adaptacyjnym modyfikowaniu tempa uczenia dla każdej wagi, ale zamiast korzystać z historycznych wartości gradientu, korzysta z wykładniczo ważonych średnich kwadratów gradientu.

**Metryki** te są wykorzystywane do oceny jakości predykcji modelu podczas procesu treningu oraz testowania.


1. Accuracy - metryka ta mierzy procent poprawnych klasyfikacji w zbiorze danych.

2. Binary accuracy - metryka ta mierzy procent poprawnych klasyfikacji dla problemów binarnych.

3. Categorical accuracy - metryka ta mierzy procent poprawnych klasyfikacji dla problemów wieloklasowych.

4. Precision - metryka ta mierzy proporcję poprawnie sklasyfikowanych przykładów pozytywnych do ogólnej liczby sklasyfikowanych pozytywnie.

5. Recall - metryka ta mierzy proporcję poprawnie sklasyfikowanych przykładów pozytywnych do ogólnej liczby prawdziwych pozytywnych.

6. F1 score - metryka ta jest średnią harmoniczną precyzji i recall, a więc uwzględnia zarówno false positives, jak i false negatives.

7. Mean squared error (MSE) - metryka ta mierzy średnią kwadratową różnicę między predykcjami a rzeczywistymi wartościami.

8. Root mean squared error (RMSE) - metryka ta mierzy pierwiastek średniej kwadratowej różnicy między predykcjami a rzeczywistymi wartościami.

9. Mean absolute error (MAE) - metryka ta mierzy średnią wartość bezwzględną różnicy między predykcjami a rzeczywistymi wartościami.

**Przykłady**

* **Klasyfikacja binarna** Funkcja na ostatniej warstwie: **sigmoid** Funkcja straty: **binary_crossentropy**
* **Wieloklasowa klasyfikacja jednoetykietowa** Funkcja na ostatniej warstwie:  **softmax** Funkcja straty: **categorical_crossentropy**
* **Wieloklasowa klasyfikacja wieloetykietowa**  Funkcja na ostatniej warstwie: **sigmoid** Funkcja straty:**binary_crossentropy**
* **Regresja dowolnych wartości** Funkcja na ostatniej warstwie:  **Brak**  Funkcja straty: **MSE**
* **Regresja wartości z zakresu od 0 do 1** Funkcja na ostatniej warstwie:  **sigmoid** Funkcja straty: n**MSE** lub **binary_crossentropy**

**Overfitting** - najprosciej (widac to na wykresach Train vs Valid) oddalanie sie wartosci metryki (np. accuracy) w procesie uczenia a w procesie testu. Co w przypadku SSN oznacza tyle że sieć na tyle dobrze nauczyła się klasyfikować zbiór treningowy, że utraciła jednocześnie zdolność do generalizowania, czyli umiejętność poprawnej klasyfikacji danych, których uprzednio nie widziała.

**Batch normalization** to technika normalizacji danych wejściowych do sieci neuronowych, która ma na celu przyspieszenie procesu uczenia oraz poprawę jakości predykcji modelu.

W skrócie, batch normalization polega na przeskalowaniu i przesunięciu wartości wejściowych każdej warstwy sieci neuronowej tak, aby średnia wartość była bliska 0 i wariancja bliska 1.

W praktyce, podczas treningu sieci neuronowej, wartości wejściowe do każdej warstwy zmieniają się wraz z każdą iteracją i batch normalization normalizuje wartości wejściowe w obrębie jednej mini-batch, czyli małej podgrupy przykładów uczących, które są przetwarzane jednocześnie. Dzięki temu, model jest mniej wrażliwy na zmiany w rozkładzie danych wejściowych, a co za tym idzie, ma większą stabilność i efektywność w procesie uczenia.

``` 
# Model with default batch normalization
model = Sequential([
    Dense(64, input_shape=(4,), activation="relu"),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(
        momentum=0.95, 
        epsilon=0.005,
        beta_initializer=RandomNormal(mean=0.0, stddev=0.05), 
        gamma_initializer=Constant(value=0.9)
    ),
    Dense(3, activation='softmax')
]);

```

**Metoda porzucania (dropout)**
*   Jest to najpopularniejszy sposób do walki z przetrenowaniem w przypadku sieci neuronowych. Dropout polega na losowym ustawieniu wychodzących krawędzi ukrytych jednostek (neuronów tworzących ukryte warstwy) na 0 przy każdej aktualizacji fazy treningu.

*   W Tensorflow dodajemy kolejne warstwy wykorzystując „Dropout” oraz definiując jaka część neuronów ma zostać zapomniana. Np. dla wartości 0.2 zostanie wylosowanych 20% połączeń do wyzerowania.

```
from tensorflow.keras.layers import Dropout
model.add(Dropout(0.2))
```
	
*   Można jeszcze warstwę dodać na samym początku podczas wczytywania danych. Dzięki temu będziemy „porzucać” część danych wejściowych.

```
model.add(Dropout(0.5, input_shape=(28,28)))
```

Jest to relatywnie prosta, ale zarazem bardzo skuteczna technika przeciwdziałania overfittingowi. Polega na losowym usuwaniu z sieci (z warstw wewnętrznych, czasami również wejściowych) pojedynczych neuronów w trakcie uczenia. Ponieważ skomplikowane sieci (a takie niewątpliwie są głębokie sieci neuronowe), szczególnie dysponujące relatywnie niewielkimi ilościami danych uczących, mają tendencję do dokładnego dopasowywania się do danych, to taki sposób deregulacji zmusza je do uczenia w sposób bardziej zgeneralizowany.

W każdej turze uczenia się każdy z neuronów jest usuwany lub zostawiany w sieci. Szanse na usunięcie definiuje się poprzez określenie prawdopodobieństwa, z jakim neuron zostanie usunięty. W oryginalnej pracy było to 50% dla każdego neuronu. Obecnie możemy samodzielnie określić to prawdopodobieństwo, a do tego dla różnych warstw może być ono różne.




**Regularyzacja**
Kolejną techniką do walki z przeuczeniem jest dokładanie do warstw kar, aby ich wartości były mniejsze. Te metody nazywamy regularyzacjami wag.

Tutaj przykład dodania regularyzacji normą L2 do warstwy gęstej:
```
from tensorflow.keras import regularizers
 
model2.add(Dense(10, activation = 'relu', 
                 kernel_regularizer=regularizers.l2(0.01)))
```



**EarlyStopping**
W tensorflow mamy do dyspozycji funkcję EarlyStopping, która monitoruje stratę na zbiorze testowym po zakończeniu każdej epoki. Jeśli strata nie maleje, wówczas trening sieci zostaje zatrzymany. Należy zdefiniować 3 podstawowe parametry ustawiając early stopping:

*   monitor – definiujemy, co chcemy 
monitorować na podstawie czego zatrzymamy proces uczenia,
*   patience – tym parametrem definiujemy liczbę epok po ilu zatrzyma się nasz model jeśli nie zaobserwujemy zmniejszania się funkcji straty,
*   verbose – a tutaj w jaki sposób będzie wyświetlana informacja o early stoping.

```
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

model2 = Sequential()
model2.add(Flatten(input_shape=(28, 28)))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(10, activation = 'softmax'))
 
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
 
EarlyStop = EarlyStopping(monitor='val_loss', 
                          patience=3,
                          verbose=1)
 
history2 = model2.fit(X_train, 
                    y_train, 
                    epochs=50,
                    verbose=1,
                    batch_size = 256,
                    validation_data = (X_val, y_val),
                    callbacks = [EarlyStop]
                   )
 
draw_curves(history2, key1='accuracy', ylim1=(0.7, 1.0), 
            key2='loss', ylim2=(0.0, 0.8))

```

# Materiały (do zapoznania się)

Materiały do zapoznania się:

* https://poloclub.github.io/cnn-explainer/
- https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53
- https://www.freecodecamp.org/news/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050/
- https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d
- https://cv-tricks.com/cnn/understand-resnet-alexnet-vgg-inception/
- https://medium.com/analytics-vidhya/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5
- http://yann.lecun.com/exdb/lenet/

Best practices to build neural network models:

* https://medium.com/@dipti.rohan.pawar/improving-performance-of-convolutional-neural-network-2ecfe0207de7
	
* https://medium.com/towards-data-science/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-i-hyper-parameter-8129009f131b
	
* https://towardsdatascience.com/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-ii-hyper-parameter-42efca01e5d7




Artykuły i snippety kodu + podstawowe zagadnienia do CNN

* Idea konwolucji: http://sciagaprogramisty.blogspot.com/2018/01/konwolucja-wstep-do-neuronowych-sieci.html

* NN architectures: https://pub.towardsai.net/main-types-of-neural-networks-and-its-applications-tutorial-734480d7ec8e?gi=28a8f0c56697 
	
* https://www.kaggle.com/code/cdeotte/how-to-choose-cnn-architecture-mnist/notebook
	
* https://medium.com/analytics-vidhya/a-guide-to-neural-network-layers-with-applications-in-keras-40ccb7ebb57a
	
* https://towardsdatascience.com/a-quick-guide-to-neural-network-optimizers-with-applications-in-keras-e4635dd1cca4
	
* https://medium.com/@andre_ye/a-quick-guide-to-neural-network-optimizers-with-applications-in-keras-e4635dd1cca4
	
* https://towardsdatascience.com/a-guide-to-neural-network-loss-functions-with-applications-in-keras-3a3baa9f71c5




Keras vs Tensorflow vs Pytorch


	
* https://karliris62.medium.com/keras-vs-tensorflow-which-one-is-the-right-fit-for-your-project-7a166fe6c64b
	
* https://www.edureka.co/blog/keras-vs-tensorflow-vs-pytorch/
	
* https://towardsdatascience.com/batch-normalization-the-greatest-breakthrough-in-deep-learning-77e64909d81d



